## **IMPORT LIBRARY**

In [2]:
import os, warnings, math
import pandas as pd

# Machine learning Model 
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report,roc_curve,auc,average_precision_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

# Deep learning Model
import tensorflow as tf

# Directory Path Saving
os.chdir('..')
dir = os.getcwd()
print(dir)

2023-02-20 12:02:18.143481: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 12:02:19.590817: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/senume/anaconda3/envs/Person_identification_Env/lib/
2023-02-20 12:02:19.590980: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/senume/anaconda3/envs/Person_identification_Env/lib/
2023-02-20

/home/senume/Project/Research/Person-Identification-using-Chebyshev-polynomial


In [3]:
warnings.filterwarnings("ignore")

## **FUNCTION DEFINITION**

In [24]:
# SVM
def SVM(X_Dataset,Y_Dataset, X_validation, Y_validation):
  # Normally, C = 1 and gamma = 'scale' are default values
  # C controls how wide the margin will be with respect to how many misclassification we are allowing
  # C is increasing --> reduce the size of the margin and fewer misclassification and vice versa

  param_grid = [
      {'C': [0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 30, 50, 80, 100],
      'gamma': ['scale', 0.5, 0.125, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001],
      'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
  ]



  optimal_params = GridSearchCV(SVC(),
                              param_grid,
                              scoring='f1_weighted',
                              cv=StratifiedKFold(5))

  optimal_params.fit(X_Dataset, Y_Dataset)
  print(optimal_params.best_params_)

  #svm
  C = optimal_params.best_params_['C']
  gamma = optimal_params.best_params_['gamma']
  kernel = optimal_params.best_params_['kernel']

  svm = SVC(C=C, gamma=gamma, kernel=kernel)
  svm.fit(X_Dataset, Y_Dataset)

  # predict the response
  svm_y_pred = svm.predict(X_validation)

  # Classification Report for the Support Vector Machine Model
  print("Classification Report : Support Vector Machine")
  classRep = classification_report(Y_validation, svm_y_pred, digits=2)
  print(classRep)

## **LOADING DATASET**

In [5]:
os.chdir(dir + '/Feature_Dataset')
L = os.listdir()

# For given Interval Type
os.chdir(L[0])

# Selecting Coefficient_Count Folder 
Coeff_Fol = os.listdir()
print("Chebyshev Coefficent",Coeff_Fol)
os.chdir(Coeff_Fol[9])

# Patient List
PatientList =  os.listdir()

# Label Specifying Each Patient
Label = {}

for n in range(0,len(PatientList)):
    Label[PatientList[n]] = n

# Spliting the Person into Family List and New Individual List
N = len(PatientList)
FamilyList_EndIndex = int(math.ceil((70/100)*N))
FamilyList = PatientList[:FamilyList_EndIndex]
NewIdividualList = PatientList[FamilyList_EndIndex:]


# Reading Dataset
print("Reading Dataset of Chebyshev Coefficient: ", Coeff_Fol[9])

Dataset = pd.DataFrame()

for name in FamilyList:
    Temp = pd.read_csv(name)
    Temp['TARGET'] = Label[name]
    Dataset = Dataset.append(Temp, ignore_index= True)


os.chdir(dir)

Chebyshev Coefficent ['20', '30', '40', '70', '50', '100', '60', '80', '90', '10']
Reading Dataset of Chebyshev Coefficient:  10


## **DATA ANALYSIS**

## **ML TRAINING**

### **SVM**

In [25]:
X = Dataset.loc[:,'CC_1':'CC_10']
Y = Dataset.loc[:,'TARGET']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size= 0.40, random_state=7823, stratify= Y)

SVM(X_train,Y_train.to_numpy(), X_validation, Y_validation.to_numpy())


{'C': 5, 'gamma': 'scale', 'kernel': 'rbf'}
Classification Report : Support Vector Machine
              precision    recall  f1-score   support

           0       0.87      0.85      0.86        53
           1       0.83      0.40      0.54        47
           2       0.86      0.92      0.89        62
           3       0.92      0.97      0.94        78
           4       0.89      1.00      0.94        58
           5       0.99      0.95      0.97       176
           6       0.71      0.82      0.76        94
           7       0.94      0.96      0.95        51
           8       0.57      0.04      0.07       100
           9       0.73      0.66      0.69        89
          10       1.00      0.98      0.99        57
          11       0.97      0.89      0.93        73
          12       0.95      0.77      0.85        52
          13       0.92      0.56      0.70        39
          14       0.57      0.78      0.66       146
          15       0.80      0.55      0.65 

### **Neural Network**